# LLM Supervised Fine-Tuning (SFT) via HuggingFace Trainer APIs
In this section, we illustrate how to use [NVIDIA FLARE](https://nvidia.github.io/NVFlare) for Large Language Models (LLMs) SFT task. Unlike the last section [Federated NLP with BERT Model](../08.1_fed_bert/federated_nlp_with_bert.ipynb) where we showed standard Pytorch training logic, it illustrates how to adapt a local training script with [HuggingFace](https://huggingface.co/) Trainer to NVFlare, which is widely used in LLM training.

We show supervised fine-tuning (SFT) using the [SFT Trainer](https://huggingface.co/docs/trl/sft_trainer) from [HuggingFace](https://huggingface.co/), together with the [Llama-3.2-1B model](https://huggingface.co/meta-llama/Llama-3.2-1B) to showcase the functionality of federated SFT, allowing HuggingFace models to be trained and adapted to federated application with NVFlare. All other models from HuggingFace can be easily adapted following the same steps.

We conducted these experiments on a single 48GB RTX 6000 Ada GPU. 

## Setup
To use Llama-3.2-1B model, please request access to the model here https://huggingface.co/meta-llama/Llama-3.2-1B and login with an access token using huggingface-cli. Git LFS is also necessary for downloads, please follow the steps in this [link](https://github.com/git-lfs/git-lfs/blob/main/INSTALLING.md).

Install required packages for training

In [ ]:
%pip install -r requirements.txt

## Data Preparation
We use one dataset to illustrate the SFT. We download and preprocess [databricks-dolly-15k](https://huggingface.co/datasets/databricks/databricks-dolly-15k).

In [ ]:
! git clone https://huggingface.co/datasets/databricks/databricks-dolly-15k /tmp/nvflare/dataset/llm/dolly

In [ ]:
! python utils/preprocess_dolly.py --training_file /tmp/nvflare/dataset/llm/dolly/databricks-dolly-15k.jsonl --output_dir /tmp/nvflare/dataset/llm/dolly

## Adaptation of Centralized Training Script to Federated
To illustrate the adaptation process, we use a single dataset with three training epochs. 
### One-call training
Centralized trainings, as the baseline for comparison with other results, are done with the following command:

In [ ]:
! python utils/hf_sft_peft.py --output_path /tmp/nvflare/workspace/llm/dolly_cen_sft --train_mode SFT

### Adaptation Step 1: iterative training
To adapt the centralized training script to federated application, we first need to "break" the single call to `trainer.train()` into iterative calls, one for each round of training.
For this purpose, we provided `utils/hf_sft_peft_iter.py` as an example, which is a modified version of `utils/hf_sft_peft.py`.
Their differences are highlighted below:

![diff](./figs/diff.png)

Note that the `trainer.train()` call is replaced by a `for` loop, and the three training epochs becomes three rounds, one epoch per round. 

This setting (1 epoch per round) is for simplicity of this example. In practice, we can set the number of rounds and local epoch per round according to the needs: e.g. 2 rounds with 2 epochs per round will result in 4 training epochs in total.

At the beginning of each round, we intentionally load a fixed model weights saved at the beginning, over-writing the previous round's saved model weights, then call `trainer.train(resume_from_checkpoint=True)` with `trainer.args.num_train_epochs` incremented by 1 so that previous logging results are not overwritten. 

The purpose of doing so is to tell if the intended weights are succesfully loaded at each round. Without using a fixed starting model, even if the model weights are not properly loaded, the training loss curve will still follow the one-call result, which is not what we want to see. 

If the intended model weights (serving as the starting point for each round, the "global model" for FL use case) is properly loaded, then we shall observe a "zig-zag" pattern in the training loss curve. This is because the model weights are reset to the same starting point at the beginning of each round, in contrast to the one-shot centralized training, where the model weights are updated continuously, and the training loss curve should follow an overall decreasing trend.

To run iterative training, we use the following command:

In [ ]:
! python utils/hf_sft_peft_iter.py --output_path /tmp/nvflare/workspace/llm/dolly_cen_sft_iter --train_mode SFT

We can observe the SFT curves with tensorboard shown below. As expected, we can see the "zig-zag" pattern in the iterative training loss curve.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /tmp/nvflare/workspace/llm

### Adaptation Step 2: federated with NVFlare
Once we have the iterative training script ready with "starting model" loading capability, it can be easily adapted to a NVFlare trainer by using [Client API](../../hello-world/ml-to-fl/pt/README.md).

The major code modifications are for receiving and returning the global model (replacing the constant one used by iterative training), as shown below:

![diff](./figs/diff_fl_1.png)
![diff](./figs/diff_fl_2.png)

### Federated Training Results
We run the federated training on a single client using NVFlare Simulator via [JobAPI](https://nvflare.readthedocs.io/en/main/programming_guide/fed_job_api.html).

In [ ]:
! python sft_job.py --client_ids dolly --data_path /tmp/nvflare/dataset/llm/ --workspace_dir /tmp/nvflare/workspace/llm/dolly_fl_sft --job_dir /tmp/nvflare/workspace/jobs/llm_hf_sft --train_mode SFT 

The SFT curves are shown below. With some training randomness, the two SFT training loss curves (centralized v.s. federated) align with each other. 

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /tmp/nvflare/workspace/llm

Now let's move on to the next section of [LLM Parameter-Efficient Fine-Tuning (PEFT)](../08.3_llm_peft/LLM_PEFT.ipynb).